In [1]:
from tensorflow.keras import models
from tensorflow.keras import layers
from google.so

In [11]:
def initialize_model():

    model = models.Sequential()
    model.add(layers.Conv2D(8, kernel_size=(4,4), activation = 'relu', input_shape = (696,520,1)))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Conv2D(16, kernel_size=(3,3), activation = 'relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Conv2D(32, kernel_size=(2,2), activation = 'relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(4, activation='relu'))

    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse',
                  optimizer='adam')

    return model

In [12]:
model = initialize_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 693, 517, 8)       136       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 346, 258, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 344, 256, 16)      1168      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 172, 128, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 171, 127, 32)      2080      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 85, 63, 32)      